# Trend Detection: Temperature vs Market Probability Changes

This notebook implements trend detection algorithms to analyze the relationship between temperature changes and Polymarket probability shifts. We'll explore:

1. **Temperature Trend Detection**: Using statistical methods to identify temperature trends
2. **Market Probability Analysis**: Analyzing probability changes in weather-related markets
3. **Correlation Analysis**: Statistical correlation between temperature trends and market movements
4. **Visualization**: Interactive plots showing trend relationships

## Data Sources
- **Weather Data**: Historical temperature data from Meteostat, Met Office, NWS
- **Market Data**: Polymarket probability data for weather-related events
- **Database**: SQLite database with normalized weather and market data

In [ ]:
# Import required libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Statistical and ML libraries
from scipy import stats
from scipy.signal import find_peaks
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Database connection and data loading
class ClimateTradeAnalyzer:
    """Main class for analyzing climate-weather market correlations"""
    
    def __init__(self, db_path="../data/climatetrade.db"):
        self.db_path = db_path
        self.weather_data = None
        self.market_data = None
        
    def load_weather_data(self, location=None, start_date=None, end_date=None):
        """Load weather data from database"""
        conn = sqlite3.connect(self.db_path)
        
        query = """
        SELECT 
            timestamp,
            location_name,
            temperature,
            temperature_min,
            temperature_max,
            humidity,
            wind_speed,
            precipitation,
            pressure
        FROM weather_data
        WHERE temperature IS NOT NULL
        """
        
        if location:
            query += f" AND location_name LIKE '%{location}%'"
        if start_date:
            query += f" AND timestamp >= '{start_date}'"
        if end_date:
            query += f" AND timestamp <= '{end_date}'"
            
        query += " ORDER BY timestamp"
        
        self.weather_data = pd.read_sql_query(query, conn)
        self.weather_data['timestamp'] = pd.to_datetime(self.weather_data['timestamp'])
        self.weather_data.set_index('timestamp', inplace=True)
        
        conn.close()
        print(f"Loaded {len(self.weather_data)} weather records")
        return self.weather_data
    
    def load_market_data(self, event_filter=None, start_date=None, end_date=None):
        """Load market data from database"""
        conn = sqlite3.connect(self.db_path)
        
        query = """
        SELECT 
            timestamp,
            event_title,
            market_id,
            outcome_name,
            probability,
            volume
        FROM polymarket_data
        WHERE probability IS NOT NULL
        """
        
        if event_filter:
            query += f" AND event_title LIKE '%{event_filter}%'"
        if start_date:
            query += f" AND timestamp >= '{start_date}'"
        if end_date:
            query += f" AND timestamp <= '{end_date}'"
            
        query += " ORDER BY timestamp"
        
        self.market_data = pd.read_sql_query(query, conn)
        self.market_data['timestamp'] = pd.to_datetime(self.market_data['timestamp'])
        self.market_data.set_index('timestamp', inplace=True)
        
        conn.close()
        print(f"Loaded {len(self.market_data)} market records")
        return self.market_data

# Initialize analyzer
analyzer = ClimateTradeAnalyzer()
print("ClimateTradeAnalyzer initialized")

In [ ]:
# Load sample data for analysis
# Note: Adjust dates and locations based on your available data

# Load weather data (London example)
weather_df = analyzer.load_weather_data(
    location="London",
    start_date="2020-01-01",
    end_date="2024-12-31"
)

# Load market data (weather-related events)
market_df = analyzer.load_market_data(
    event_filter="weather",
    start_date="2020-01-01",
    end_date="2024-12-31"
)

print("\nData Overview:")
print("=" * 50)
if weather_df is not None and not weather_df.empty:
    print(f"Weather data shape: {weather_df.shape}")
    print(f"Weather date range: {weather_df.index.min()} to {weather_df.index.max()}")
    print(f"Locations: {weather_df['location_name'].unique()}")

if market_df is not None and not market_df.empty:
    print(f"\nMarket data shape: {market_df.shape}")
    print(f"Market date range: {market_df.index.min()} to {market_df.index.max()}")
    print(f"Events: {market_df['event_title'].nunique()} unique events")
    print(f"Markets: {market_df['market_id'].nunique()} unique markets")

## 1. Temperature Trend Detection Algorithms

We'll implement several trend detection methods:
- **Linear Regression Trend**: Simple slope-based trend detection
- **Moving Average Crossover**: Short-term vs long-term trend identification
- **Seasonal Decomposition**: Extracting trend from seasonal patterns
- **Change Point Detection**: Identifying significant trend changes

In [ ]:
class TrendDetector:
    """Class for detecting trends in time series data"""
    
    def __init__(self):
        self.scaler = StandardScaler()
    
    def linear_trend(self, data, window_days=30):
        """Calculate linear trend using rolling regression"""
        trends = []
        
        for i in range(window_days, len(data)):
            window = data.iloc[i-window_days:i]
            if len(window.dropna()) < window_days * 0.8:  # Require 80% data
                trends.append(np.nan)
                continue
                
            # Linear regression on window
            x = np.arange(len(window))
            y = window.values
            mask = ~np.isnan(y)
            
            if np.sum(mask) < 2:
                trends.append(np.nan)
                continue
                
            slope, intercept, r_value, p_value, std_err = stats.linregress(x[mask], y[mask])
            trends.append(slope)
        
        # Pad with NaN for initial window
        trends = [np.nan] * (window_days - 1) + trends
        return pd.Series(trends, index=data.index)
    
    def moving_average_crossover(self, data, short_window=7, long_window=30):
        """Detect trend changes using moving average crossover"""
        short_ma = data.rolling(window=short_window, center=True).mean()
        long_ma = data.rolling(window=long_window, center=True).mean()
        
        # Trend signal: 1 for uptrend, -1 for downtrend, 0 for neutral
        trend_signal = np.where(short_ma > long_ma, 1, 
                               np.where(short_ma < long_ma, -1, 0))
        
        return pd.Series(trend_signal, index=data.index)
    
    def seasonal_trend(self, data, period=365):
        """Extract trend component using seasonal decomposition"""
        try:
            # Resample to daily if needed
            if isinstance(data.index, pd.DatetimeIndex):
                daily_data = data.resample('D').mean()
            else:
                daily_data = data
            
            # Remove NaN values for decomposition
            clean_data = daily_data.dropna()
            
            if len(clean_data) < period * 2:
                print(f"Warning: Not enough data for seasonal decomposition. Need at least {period * 2} points.")
                return pd.Series([np.nan] * len(data), index=data.index)
            
            decomposition = seasonal_decompose(clean_data, model='additive', period=period)
            trend = decomposition.trend
            
            # Reindex to original data
            trend_reindexed = trend.reindex(data.index, method='nearest')
            return trend_reindexed
            
        except Exception as e:
            print(f"Error in seasonal decomposition: {e}")
            return pd.Series([np.nan] * len(data), index=data.index)
    
    def detect_change_points(self, data, threshold=2.0):
        """Detect significant change points in the trend"""
        # Calculate rolling standard deviation
        rolling_std = data.rolling(window=30).std()
        
        # Calculate z-score of changes
        diff = data.diff()
        z_scores = np.abs((diff - diff.mean()) / diff.std())
        
        # Identify change points where z-score exceeds threshold
        change_points = z_scores > threshold
        
        return change_points

# Initialize trend detector
trend_detector = TrendDetector()
print("TrendDetector initialized")

In [ ]:
# Apply trend detection to temperature data
if weather_df is not None and not weather_df.empty:
    print("Applying trend detection algorithms to temperature data...")
    
    # Extract temperature series
    temp_series = weather_df['temperature'].dropna()
    
    if len(temp_series) > 0:
        # 1. Linear trend detection
        linear_trends = trend_detector.linear_trend(temp_series, window_days=30)
        
        # 2. Moving average crossover
        ma_signals = trend_detector.moving_average_crossover(temp_series)
        
        # 3. Seasonal trend
        seasonal_trends = trend_detector.seasonal_trend(temp_series)
        
        # 4. Change point detection
        change_points = trend_detector.detect_change_points(temp_series)
        
        print("Trend detection completed!")
        print(f"Linear trends calculated: {linear_trends.count()} points")
        print(f"MA signals: {ma_signals.value_counts().to_dict()}")
        print(f"Change points detected: {change_points.sum()}")
    else:
        print("No temperature data available for trend analysis")
else:
    print("No weather data available")

In [ ]:
# Visualize temperature trends
if 'temp_series' in locals() and len(temp_series) > 0:
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # Plot 1: Original temperature data with trends
    axes[0].plot(temp_series.index, temp_series.values, 'b-', alpha=0.7, label='Temperature')
    if 'seasonal_trends' in locals():
        axes[0].plot(seasonal_trends.index, seasonal_trends.values, 'r-', linewidth=2, label='Seasonal Trend')
    axes[0].set_title('Temperature Data with Seasonal Trend')
    axes[0].set_ylabel('Temperature (°C)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Linear trends
    if 'linear_trends' in locals():
        valid_trends = linear_trends.dropna()
        axes[1].plot(valid_trends.index, valid_trends.values, 'g-', label='Linear Trend Slope')
        axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.5, label='No Trend')
        axes[1].set_title('Linear Trend Detection (30-day windows)')
        axes[1].set_ylabel('Trend Slope')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
    
    # Plot 3: Moving average signals
    if 'ma_signals' in locals():
        colors = ['red' if x == -1 else 'green' if x == 1 else 'gray' for x in ma_signals.values]
        axes[2].scatter(ma_signals.index, ma_signals.values, c=colors, alpha=0.6, s=20)
        axes[2].set_title('Moving Average Crossover Signals')
        axes[2].set_ylabel('Trend Signal')
        axes[2].set_yticks([-1, 0, 1])
        axes[2].set_yticklabels(['Downtrend', 'Neutral', 'Uptrend'])
        axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\nTrend Analysis Summary:")
    print("=" * 40)
    if 'linear_trends' in locals():
        positive_trends = (linear_trends > 0).sum()
        negative_trends = (linear_trends < 0).sum()
        print(f"Positive trends: {positive_trends}")
        print(f"Negative trends: {negative_trends}")
        print(f"Average trend slope: {linear_trends.mean():.6f}")
    
    if 'change_points' in locals():
        print(f"Significant change points: {change_points.sum()}")
else:
    print("No temperature data available for visualization")

## 2. Market Probability Change Analysis

Now let's analyze how market probabilities change over time, particularly for weather-related events.

In [ ]:
# Analyze market probability changes
if market_df is not None and not market_df.empty:
    print("Analyzing market probability changes...")
    
    # Group by market and calculate probability changes
    market_groups = market_df.groupby('market_id')
    
    probability_changes = []
    
    for market_id, group in market_groups:
        if len(group) > 1:
            # Sort by timestamp
            group_sorted = group.sort_index()
            
            # Calculate probability changes
            prob_changes = group_sorted['probability'].diff()
            
            # Calculate volatility (standard deviation of changes)
            volatility = prob_changes.std()
            
            # Calculate trend in probability
            prob_trend = trend_detector.linear_trend(group_sorted['probability'], window_days=7)
            
            probability_changes.append({
                'market_id': market_id,
                'event_title': group['event_title'].iloc[0],
                'data_points': len(group),
                'probability_range': group['probability'].max() - group['probability'].min(),
                'volatility': volatility,
                'avg_probability': group['probability'].mean(),
                'final_probability': group_sorted['probability'].iloc[-1],
                'probability_trend': prob_trend.mean() if not prob_trend.empty else 0
            })
    
    # Convert to DataFrame
    prob_change_df = pd.DataFrame(probability_changes)
    
    print(f"Analyzed {len(probability_changes)} markets")
    print("\nTop 10 most volatile markets:")
    if not prob_change_df.empty:
        top_volatile = prob_change_df.nlargest(10, 'volatility')
        for _, row in top_volatile.iterrows():
            print(f"{row['market_id'][:8]}...: {row['event_title'][:50]}... (Vol: {row['volatility']:.4f})")
else:
    print("No market data available for analysis")

## 3. Temperature-Market Correlation Analysis

Now we'll correlate temperature trends with market probability changes.

In [ ]:
# Correlation analysis between temperature and market data
def analyze_temperature_market_correlation(weather_df, market_df, max_lag_days=7):
    """Analyze correlation between temperature trends and market probability changes"""
    
    if weather_df is None or market_df is None or weather_df.empty or market_df.empty:
        print("Insufficient data for correlation analysis")
        return None
    
    correlations = []
    
    # Get temperature data
    temp_data = weather_df['temperature'].dropna()
    
    # Analyze each market
    for market_id in market_df['market_id'].unique():
        market_subset = market_df[market_df['market_id'] == market_id]
        prob_data = market_subset['probability'].dropna()
        
        if len(prob_data) < 10:  # Skip markets with too few data points
            continue
        
        # Find overlapping time period
        start_date = max(temp_data.index.min(), prob_data.index.min())
        end_date = min(temp_data.index.max(), prob_data.index.max())
        
        if start_date >= end_date:
            continue
        
        # Align data to common time period
        temp_aligned = temp_data[(temp_data.index >= start_date) & (temp_data.index <= end_date)]
        prob_aligned = prob_data[(prob_data.index >= start_date) & (prob_data.index <= end_date)]
        
        if len(temp_aligned) < 10 or len(prob_aligned) < 10:
            continue
        
        # Resample to daily frequency for correlation
        temp_daily = temp_aligned.resample('D').mean()
        prob_daily = prob_aligned.resample('D').mean()
        
        # Calculate correlations with different lags
        for lag in range(max_lag_days + 1):
            if lag == 0:
                temp_shifted = temp_daily
                prob_shifted = prob_daily
            else:
                temp_shifted = temp_daily.shift(lag)
                prob_shifted = prob_daily
            
            # Remove NaN values
            combined = pd.concat([temp_shifted, prob_shifted], axis=1, keys=['temp', 'prob']).dropna()
            
            if len(combined) < 5:
                continue
            
            # Calculate correlation
            corr = combined['temp'].corr(combined['prob'])
            
            if not np.isnan(corr):
                correlations.append({
                    'market_id': market_id,
                    'event_title': market_subset['event_title'].iloc[0],
                    'lag_days': lag,
                    'correlation': corr,
                    'data_points': len(combined),
                    'temp_mean': combined['temp'].mean(),
                    'prob_mean': combined['prob'].mean()
                })
    
    return pd.DataFrame(correlations)

# Perform correlation analysis
correlation_results = analyze_temperature_market_correlation(weather_df, market_df)

if correlation_results is not None and not correlation_results.empty:
    print(f"\nCorrelation Analysis Results:")
    print(f"Total correlations calculated: {len(correlation_results)}")
    
    # Show strongest correlations
    print("\nTop 10 strongest correlations:")
    top_corr = correlation_results.nlargest(10, 'correlation')
    for _, row in top_corr.iterrows():
        print(f"{row['correlation']:.3f} (lag: {row['lag_days']}d): {row['event_title'][:60]}...")
    
    print("\nTop 10 strongest negative correlations:")
    bottom_corr = correlation_results.nsmallest(10, 'correlation')
    for _, row in bottom_corr.iterrows():
        print(f"{row['correlation']:.3f} (lag: {row['lag_days']}d): {row['event_title'][:60]}...")
else:
    print("No correlation results available")

In [ ]:
# Visualize correlation results
if correlation_results is not None and not correlation_results.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot 1: Correlation distribution
    axes[0, 0].hist(correlation_results['correlation'], bins=20, alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('Distribution of Temperature-Market Correlations')
    axes[0, 0].set_xlabel('Correlation Coefficient')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Correlation by lag
    lag_corr = correlation_results.groupby('lag_days')['correlation'].agg(['mean', 'std', 'count'])
    axes[0, 1].errorbar(lag_corr.index, lag_corr['mean'], yerr=lag_corr['std'], 
                       marker='o', capsize=5, label='Mean ± Std')
    axes[0, 1].set_title('Average Correlation by Lag Days')
    axes[0, 1].set_xlabel('Lag (Days)')
    axes[0, 1].set_ylabel('Average Correlation')
    axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()
    
    # Plot 3: Strongest correlations scatter
    strong_corr = correlation_results[abs(correlation_results['correlation']) > 0.5]
    if not strong_corr.empty:
        scatter = axes[1, 0].scatter(strong_corr['temp_mean'], strong_corr['prob_mean'], 
                                    c=strong_corr['correlation'], cmap='coolwarm', 
                                    alpha=0.7, s=50)
        axes[1, 0].set_title('Strong Correlations: Temperature vs Market Probability')
        axes[1, 0].set_xlabel('Average Temperature (°C)')
        axes[1, 0].set_ylabel('Average Market Probability')
        plt.colorbar(scatter, ax=axes[1, 0], label='Correlation')
        axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Correlation strength vs data points
    axes[1, 1].scatter(correlation_results['data_points'], abs(correlation_results['correlation']), 
                       alpha=0.6, s=30)
    axes[1, 1].set_title('Correlation Strength vs Sample Size')
    axes[1, 1].set_xlabel('Number of Data Points')
    axes[1, 1].set_ylabel('Absolute Correlation')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\nCorrelation Summary Statistics:")
    print("=" * 40)
    print(correlation_results['correlation'].describe())
    
    strong_positive = (correlation_results['correlation'] > 0.5).sum()
    strong_negative = (correlation_results['correlation'] < -0.5).sum()
    
    print(f"\nStrong correlations (|r| > 0.5):")
    print(f"Positive: {strong_positive}")
    print(f"Negative: {strong_negative}")
    print(f"Total: {strong_positive + strong_negative}")
else:
    print("No correlation data available for visualization")

## 4. Advanced Trend Analysis

Let's implement more sophisticated trend analysis techniques.

In [ ]:
# Advanced trend analysis with statistical tests
def advanced_trend_analysis(data, name="Series"):
    """Perform advanced statistical trend analysis"""
    
    if data is None or len(data.dropna()) < 20:
        print(f"Insufficient data for advanced analysis of {name}")
        return None
    
    clean_data = data.dropna()
    
    results = {
        'series_name': name,
        'data_points': len(clean_data),
        'date_range': f"{clean_data.index.min()} to {clean_data.index.max()}",
        'mean': clean_data.mean(),
        'std': clean_data.std(),
        'min': clean_data.min(),
        'max': clean_data.max()
    }
    
    # Stationarity tests
    try:
        # Augmented Dickey-Fuller test
        adf_result = adfuller(clean_data)
        results['adf_statistic'] = adf_result[0]
        results['adf_pvalue'] = adf_result[1]
        results['adf_stationary'] = adf_result[1] < 0.05
    except:
        results['adf_statistic'] = None
        results['adf_pvalue'] = None
        results['adf_stationary'] = None
    
    try:
        # KPSS test
        kpss_result = kpss(clean_data)
        results['kpss_statistic'] = kpss_result[0]
        results['kpss_pvalue'] = kpss_result[1]
        results['kpss_stationary'] = kpss_result[1] >= 0.05
    except:
        results['kpss_statistic'] = None
        results['kpss_pvalue'] = None
        results['kpss_stationary'] = None
    
    # Trend analysis
    try:
        x = np.arange(len(clean_data))
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, clean_data.values)
        
        results['linear_slope'] = slope
        results['linear_intercept'] = intercept
        results['linear_r_squared'] = r_value**2
        results['linear_p_value'] = p_value
        results['linear_trend_significant'] = p_value < 0.05
        
        # Calculate trend direction and strength
        if abs(slope) < 1e-6:
            results['trend_direction'] = 'flat'
        elif slope > 0:
            results['trend_direction'] = 'increasing'
        else:
            results['trend_direction'] = 'decreasing'
            
        results['trend_strength'] = abs(slope) * len(clean_data) / clean_data.std()
        
    except:
        results['linear_slope'] = None
        results['trend_direction'] = None
        results['trend_strength'] = None
    
    # Volatility analysis
    try:
        returns = clean_data.pct_change().dropna()
        results['volatility'] = returns.std() * np.sqrt(252)  # Annualized volatility
        results['skewness'] = stats.skew(clean_data)
        results['kurtosis'] = stats.kurtosis(clean_data)
    except:
        results['volatility'] = None
        results['skewness'] = None
        results['kurtosis'] = None
    
    return results

# Perform advanced analysis
print("Performing advanced trend analysis...")

analysis_results = []

if weather_df is not None and not weather_df.empty:
    temp_analysis = advanced_trend_analysis(weather_df['temperature'], "Temperature")
    if temp_analysis:
        analysis_results.append(temp_analysis)

if market_df is not None and not market_df.empty:
    # Analyze average probability across all markets
    avg_prob = market_df.groupby(market_df.index)['probability'].mean()
    prob_analysis = advanced_trend_analysis(avg_prob, "Market Probability")
    if prob_analysis:
        analysis_results.append(prob_analysis)

# Display results
if analysis_results:
    results_df = pd.DataFrame(analysis_results)
    print("\nAdvanced Trend Analysis Results:")
    print("=" * 50)
    
    for _, row in results_df.iterrows():
        print(f"\n{row['series_name']}:")
        print(f"  Data points: {row['data_points']}")
        print(f"  Date range: {row['date_range']}")
        print(f"  Mean: {row['mean']:.4f}")
        print(f"  Trend: {row['trend_direction']} (slope: {row.get('linear_slope', 'N/A')})")
        if row.get('linear_p_value') is not None:
            print(f"  Trend significance: {'Significant' if row['linear_trend_significant'] else 'Not significant'} (p={row['linear_p_value']:.4f})")
        if row.get('volatility') is not None:
            print(f"  Volatility: {row['volatility']:.4f}")
else:
    print("No data available for advanced analysis")

## 5. Summary and Key Findings

This notebook has implemented comprehensive trend detection algorithms for analyzing the relationship between temperature changes and Polymarket probability shifts. Here's what we've accomplished:

### Key Features Implemented:
1. **Multiple Trend Detection Methods**: Linear regression, moving averages, seasonal decomposition, and change point detection
2. **Market Probability Analysis**: Volatility analysis and trend detection for weather-related markets
3. **Correlation Analysis**: Statistical correlation between temperature trends and market movements with lag analysis
4. **Advanced Statistical Tests**: Stationarity tests (ADF, KPSS) and comprehensive trend analysis
5. **Interactive Visualizations**: Multiple plots showing trends, correlations, and patterns

### Potential Applications:
- **Trading Strategies**: Identify weather-market arbitrage opportunities
- **Risk Assessment**: Evaluate market sensitivity to weather patterns
- **Predictive Modeling**: Forecast market movements based on weather trends
- **Research**: Study climate change impacts on financial markets

### Next Steps:
1. **Real-time Integration**: Connect with live weather and market data streams
2. **Machine Learning**: Implement predictive models using the trend features
3. **Geospatial Analysis**: Analyze regional weather-market relationships
4. **Portfolio Optimization**: Develop weather-hedged trading strategies

### Data Requirements:
- Historical weather data (temperature, precipitation, wind, etc.)
- Polymarket probability data for weather-related events
- Sufficient time overlap between weather and market data
- Clean, normalized data in SQLite database

This analysis framework provides a solid foundation for understanding and exploiting weather-market correlations in the Polymarket ecosystem.